# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = '/data/models/THUDM/chatglm3-6b'

/home/ubuntu/anaconda3/envs/liweirong-quickstart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map="auto",
                                  trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.25s/it]


In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，也是八宫图之一，其卦象由三个阳爻夹一个阴爻构成，象征着天、云和雷。乾卦的卦辞是“元、亨、利、贞”，意味着“创造者、通、顺利、正”，它代表着创造、积极、正义和力量等象征。

在八宫图中，乾卦位于北方，与事业、努力、领导、坚定等有关。乾卦所代表的能量具有强烈的领导欲望和自我主张，它鼓励人们积极向前，勇往直前，不畏艰难险阻，创造新的成果。

在易经中，乾卦所代表的阳爻象征着阳刚之气，阴爻则象征着阴柔之气。两者相互配合，形成一种刚柔并济的状态。因此，乾卦所代表的能量既具有阳刚之气的坚定和力量，也具有阴柔之气的灵活和适应性。


In [9]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是八卦之一，其卦象由两个阴爻夹一个阳爻构成，象征着地、水、云和风。师卦的卦辞是“元、亨、利、贞”，意味着“创造者、通、顺利、正”，它代表着创造、积极、正义和力量等象征。

在八宫图中，地水师卦位于北方，与学习、智慧、知识、信息等有关。地水师卦所代表的能量具有智慧、聪明和灵活的特点，它鼓励人们不断学习，不断探索，积极获取新的知识和信息，以适应不断变化的环境。

在易经中，阳爻象征着阳刚之气，阴爻则象征着阴柔之气。地水师卦中阴阳爻的配合，形成一种刚柔并济的状态。因此，地水师卦所代表的能量既具有阳刚之气的坚定和力量，也具有阴柔之气的灵活和适应性。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [12]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [13]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [14]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，包含六爻，由上善下良的风德运行。

乾卦的初爻为阳，中爻为阳，末爻为阳，上爻为阳，中爻为阳，末爻为阳，这六个阳爻构成阳极，因此乾卦始终为阳。因此，乾卦的含义是：阳刚之力量，能充实运行于万事万物之中。

在《易经》中，乾卦的卦辞为：“元、亨、利、贞”，意味着元、始、亨、通，利、谐、贞，代表着元始、运行、和谐、正统。乾卦的时运是利于运行，且运行的速度快且远。

乾卦代表天，象征刚健强劲的特性。它预示着充实运行于万事万物之中，且速度快且远。在《易经》中，乾卦的卦辞提醒人们要维系天刚强的特性，且速行千里，警惕 dance（跳舞）之患。

乾卦的卦象是由六个阳爻构成，代表着天，意味着刚健强劲的特性。它预示着充实运行于万事万物之中，且速度快且远。在《易经》中，乾卦的卦辞提醒人们要维系天刚强的特性，且速行千里，警惕 dance（跳舞）之患。

总结起来，乾卦是周易中的一卦，代表天，包含六爻，由上善下良的风德运行。它预示着刚健强劲的特性，且速行千里，但需警惕 dance（跳舞）之患。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，包含六条线，象征天象。它由两个阳爻（代表天）和一条阴爻（代表地）组成，预示着万物创始时的天象。在卜问中，这个卦象预示着吉祥。在行为上， it 意味着积极前进，冒险前进，注意安全。在决策上，它预示着吉祥顺利，有利于君子。

乾卦的核心哲学是：“天行健，君子以自强不息；天行健，君子以明德自修。” 这意味着君子应当效法天的运行，不断自强，修养德行。在实际生活中，这意味着君子应该积极前进，勇于冒险，同时注意安全。在决策上，他们应该追求吉祥，有利于自身和他人。


In [15]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
地水师卦是周易中的一卦，由坤卦（地）和坎卦（水）组合而成。在卜卦时，指数已定，只待省力，则可获得美果。这种卦象象征着地面上的河流，说明事物变化的过程和力量，以及人的品性和行动。

在卜卦时，地水师卦意味着人的品性中充满了柔顺、温和、宽柔、刚毅等不同的特质，需要人们不断努力，积极进取，以实现自己的目标和愿望。地面上的河流虽然柔顺，但流速却很快，因此，人们需要善于控制自己，以保持稳健的速度前进。

地水师卦提醒人们，在人生道路上要善于适应变化，随机应变，不要固执己见，要具备谦逊、谦逊的态度，才能在各种困难和挑战中获得成功。在解卦时，需要结合具体的问题和情况，分析卦象，从而得出合适的答案。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦

 地水师卦是周易中的一卦，由地（坤）卦和坎（水）卦组合而成。地卦代表大地的形势，象征着顺顺利利，而水卦则代表流动的水，暗示着变化和不安。

在卜筮时，先取地卦，后取水卦，最后取师卦。这一卦象代表吉祥，具有柔顺柔顺的特点，代表吉利的事情。在解卦时，象辞描述了吉利的事情，提醒人们要谦逊、谨慎。

卜筮时，地水师卦代表吉凶 Depending on the context of the query, the卜筮结果 may represent good or bad luck.

* If the query is related to career or business, the卜筮 result may indicate a successful and prosperous career, with opportunities for advancement and success.
* If the query relates to relationships, the result may indicate a harmonious and loving relationship, with trust and understanding.
* If the query relates to finances, the result may indicate financial stability and prosperity, with opportunities for gro

In [16]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
天水讼卦是周易中的一卦，由上坎卦（水）和下乾卦（天）组成，代表天水相交，预示着尽管有贤人，但难以rescind their sentences（ reversing their statements），从而导致诉讼不断。在周易中，这是一卦象征着一个艰难的局势，需要人们保持冷静、谨慎行事。在卜卦时，这一卦提醒人们要注意分辨黑白，辨明是非，避免陷入诉讼之中。

在Meta解码中，代表卜卦者将面临选择，需要做出明智的决策，同时要谨慎处理问题。在互联网上，人们需要对于各种信息进行筛选，避免被虚假信息所迷惑。在人际交往中，人们需要保持冷静，避免因为情绪化而做出错误的决策。

对于这一卦，卜卦者需要如同《象辞》中所说：“的天水谣》，其色取天，其气取水。” 意味着要顺应天时地利，以智慧为武器，以和平的方式解决争端。对于现状，虽然有困难，但仍有希望，只要努力，仍有机会。

对于运势，上天给予你极大的压力和挑战，但你可以通过冷静、理智和勇敢的方式来应对。即使遇到困难和挫折，也要保持冷静，以平和的心态面对，依然有望取得成功。

总结起来，天猫诉卦告訴你，面对困难时，需要保持冷静、理智，以智慧和勇气去解决问题，同时需要谨慎待人，避免因为情绪化而做出错误的决策。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦


的天水讼卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表诉讼和争斗的卦象。在《易经》中，讼卦是一个极具深意的卦象，提示着人们要警惕矛盾和冲突，要和平解决，避免战争。

讼卦的卦象特点为：上坎下坤。在卜卦时，需要关注上卦坎卦的动态，下卦坤卦的稳定。在解卦时，要结合上卦坎卦的流动和下卦坤卦的静止，注意培养德行，避免矛盾和争斗。

讼卦的核心哲学是：矛盾和冲突可以带来伤害，因此要警惕并和平解决。在处理矛盾和冲突时，要善于用礼义和智慧，以避免战争和灾难。

讼卦的启示是：谦逊、谦逊和宽容是解决矛盾和冲突的关键。要重视德行的培养，以避免矛盾和争斗，建立和谐人际关系。

讼卦的时运方面，一般来说，在事业和 business 上可能会遇到困难和挑战，需要谨慎处理，避免过于冲动。同时，在处理问题和争议时，要冷静思考，避免冲动行为。

在事业和 business 上，讼卦提醒要注重沟通和协作，以避免矛盾和冲突。在处理问题和争议时，要善于运用智慧和

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [17]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [18]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [19]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，包含六爻，分为阳爻和阴爻两种。上卦为阳爻，下卦为阴爻，中卦为阳爻，相见为吉，不相见为忧，为天。上卦代表阳刚之象，下卦代表阴柔之象，相互配合，刚柔相济，代表天地运行的规律，预示吉利的预兆。

乾卦的卦辞是：天行健，君子以自强不息；天行健，君子以自强不息。这个卦意味着天时地利人和都处于运行的状态，时机成熟，人力努力，事情顺利。因此，这是一个充满活力和决心的卦象。

在卜问中，刚毅的卦象被强调，表示充满精力、意志力和决心的重要性。在商业领域，这个卦象预示着成功， index


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，包含六条线，象征天象。它由两个阳爻（代表天）和一条阴爻（代表地）组成，预示着万物创始时的天象。在卜问中，这个卦象预示着吉祥。在行为上， it 意味着积极前进，冒险前进，注意安全。在决策上，它预示着吉祥顺利，有利于君子。

乾卦的核心哲学是：“天行健，君子以自强不息；天行健，君子以明德自修。” 这意味着君子应当效法天的运行，不断自强，修养德行。在实际生活中，这意味着君子应该积极前进，勇于冒险，同时注意安全。在决策上，他们应该追求吉祥，有利于自身和他人。


In [20]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易卦象中的一枚卦，由两个“地”字叠加而成，代表水在地下流动，犹如地中的溪水般波澜涌动。卦象代表 Context，沸点代表动乱，预示着变化即将来临。在《易经》中，地水师卦被解释为“观于水之贞疾”，强调变化无常，动乱即将到来，必须保持警惕。

在解卦时，地水师卦提示着um000080，um000080，um000080。解卦表示，对于地水师卦，要重视变化，警惕动乱，要积极应对，保持冷静，以避免不利影响。

地水师卦的核心哲学是：观于水之贞疾，对于地水师卦，要警惕变化，积极应对，以避免不利影响。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是周易中的一卦，由地（坤卦）与水（坎卦）相合而成。地象征大地的特性，具有承载、包容、适应的特点。水象征流动、变化的特点，具有流动、适应、变化的能力。地水师卦预示着 Compassion (仁) 与 Change (变) 之间的相互关系，表明了 Compassion (仁) 必须适应变化，而变化则需要依赖于 Compassion (仁)。

在解卦时，地水师卦的卦象为：上坤地，中坎水，下坤地。上坤地代表地，象征大地的特性，具有承载、包容、适应的特点。中坎水代表水，具有流动、变化的特点，具有流动、适应、变化的能力。下坤地代表地，再次强调了大地的特性。

地水师卦的时运为：地水时运为：地水胜时运，时运为：地水胜时运。这意味着在不同的时期，地水师卦的运势会有所不同。在好运时，地水师卦会得到支持和发展，而在坏事时，地水师卦可能会受到抑制和困扰。

地水师卦的运势受到地水胜时运的影响，好运时，地水师卦会得到支持和发展，坏事时，地水师卦可能会受到抑制和困扰。


In [21]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么样子

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [22]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240214_180729"
peft_model_path = f"models{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [23]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [24]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
{'name': '震动卦，地在上，火在下，名为兑。兑卦是周易中的一卦，由两个兑字叠加而成，代表兑泽，是兑卦的象征。兑泽之卦辞为：“兑，泽的坚固和高远，君子观此卦象，从而占卜大吉。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊有礼之态度。兑卦核心观念是：君子观此卦象，应当紧抓住机会，切勿错过名利之机会。同时，也应保持谦虚有礼之态度，才能取得成功。', 'content': '\n在事业方面，兑卦代表事业之成功，但需谨慎珍惜，务必保持谦虚有礼之态度。在经商方面，也需谨慎立方，机遇和风险并存，必须谨慎把握。而在婚姻方面，兑卦代表婚姻之美好，但亦需保持谦虚有礼之态度。在决策方面，兑卦提示必须保持谦虚有礼之态度，方能取得成功，但也需警惕风险和危险。\n\n兑卦的核心哲学是：经商、婚恋、经商皆有利，但务必谨慎行事，保持谦虚有礼之态度。同时也需警惕风险和危险。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [25]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
坤为地，地为坤卦， here is a卦 composed of earth and water, which represents the earth-bound and earthy aspects of life. In the original configuration, the upper卦代表地，下卦代表水，象征着大地的宁静与宁静的结合。地水师卦是一个卦象，代表着大地上的和谐与平衡，以及人与自然的和谐共处。在这个卦象中，预示着大吉大利，但需要谨慎行事，以便获得美好结局。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。在这个卦中，地象征着 stabilizing the root of the matter, while the water symbolizes the creative force. The卦象 represents a situation where the root of the matter is stabilized, but the creative force is not being utilized effectively.

This卦象 suggests that the problem needs to be addressed by focusing on the root of the matter and utilizing the creative force. It also suggests that a proper response to the situation requires patience and persistence, as well as a willingness to wait for the right moment to act.

地水师卦的核心哲学是: 宜急寻根本，次第行动，等待时机，不可急进。在事业、经商、求名、婚恋和决策等方面，都需要注意这一点。在处理事务时，应先从 stabilizing the root of the matter入手，然后积极利用 creative forc

In [26]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么卦象,预示着吉凶br>br> 讼卦是周易中的一卦,由上卦乾（天）和下卦坎（水）组成。这一卦象代表着天与水相交的画面,寓意着君子应当像天一样高洁，像水一样低逊。君子观此卦象,应当以高洁的品质作为标准，以低逊的态度对待他人，这样就能够得到美好的结果。然而，君子应当避免与小人的冲突，以免招来不幸。在事业上，务必 heighten品质，发挥优点，同时要低逊待人，这样才能够获得良好的结果。对于经商，必须以诚信为本，积极进取，才能够获得美好的收益。总体而言，这一卦象预示着君子若涉涉水,则必得 clean（清洁）归正。
